## Shallow Neural Network with PyTorch Autograd

## Data preparation & Import

In [52]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [53]:
## XOR data (NumPy)
x_seeds = np.array([(0,0),(1,0),(0,1),(1,1)], dtype=np.float32)
y_seeds = np.array([0,1,1,0])

N = 1000
idxs = np.random.randint(0,4,N)

X = x_seeds[idxs]
Y = y_seeds[idxs]

X += np.random.normal(scale = 0.25, size = X.shape)
print("X shape : " + str(X.shape) + "\nY shape : " + str(Y.shape))

X shape : (1000, 2)
Y shape : (1000,)


## Model

In [54]:
class shallow_neural_network():
    def __init__(self, num_input_features, num_hiddens):
        self.num_input_features = num_input_features
        self.num_hiddens = num_hiddens

        self.W1 = torch.randn((num_hiddens, num_input_features),requires_grad=True)
        self.b1 = torch.randn(num_hiddens,requires_grad=True)
        self.W2 = torch.randn(num_hiddens,requires_grad=True)
        self.b2 = torch.randn(1,requires_grad=True)
        
        self.tanh = torch.nn.Tanh()
        self.sigmoid = torch.nn.Sigmoid()

    def predict(self,x):
        ## x.shape == (2,) , z1.shape == (3,)
        z1 = torch.matmul(self.W1,x) + self.b1
        a1 = self.tanh(z1)   # (3,)
        z2 = torch.matmul(self.W2,a1) + self.b2
        # z2.shape == (1,)
        a2 = self.sigmoid(z2)
        return a2

## Training

In [55]:
def train(X,Y,model,lr = 0.1):
    # m training examples
    m = len(X)

    cost = 0.0
    for x,y in zip(X,Y):
        x_torch = torch.from_numpy(x)
        a2 = model.predict(x_torch)
        
        loss = 0.0
        if y == 1:
            loss -= torch.log(a2+0.0001)
        else: 
            loss -= torch.log(1.0001-a2)

        loss.backward()
        cost += loss.item()

    with torch.no_grad():
        model.W1 -= lr * model.W1.grad/m
        model.b1 -= lr * model.b1.grad/m
        model.W2 -= lr * model.W2.grad/m
        model.b2 -= lr * model.b2.grad/m
    
    model.W1.requires_grad = True
    model.b1.requires_grad = True
    model.W2.requires_grad = True
    model.b2.requires_grad = True
    
    return cost/m

In [56]:
model = shallow_neural_network(2,3)

for epoch in range(100):
    cost = train(X,Y,model,1.0)
    if epoch%10 == 0:
        print(epoch,cost)

0 0.7427808288633824
10 0.6469942117333413
20 0.2550292861610651
30 0.22157075064152792
40 0.2201689505310353
50 0.26344249155037747
60 0.27912642528675385
70 0.2341277286703844
80 0.28338343895208257
90 0.25614442194087494


## Testing

In [57]:
print(model.predict(torch.Tensor((0,0))))
print(model.predict(torch.Tensor((0,1))))
print(model.predict(torch.Tensor((1,0))))
print(model.predict(torch.Tensor((1,1))))

tensor([1.4492e-21], grad_fn=<SigmoidBackward>)
tensor([1.], grad_fn=<SigmoidBackward>)
tensor([0.9290], grad_fn=<SigmoidBackward>)
tensor([5.2646e-22], grad_fn=<SigmoidBackward>)
